# Preamble

In [1]:
import numpy as np
import itertools

Some Matrix functions:

In [2]:
def dag(M):
    return np.conjugate(M.T)
def matInv(M):
    return np.linalg.inv(M)
def is_eq(A,B):
    return np.allclose(A,B, rtol=1e-05, atol=1e-08)

In [3]:
om = np.exp(1.j*2*np.pi/3)
print(om)

(-0.49999999999999983+0.8660254037844387j)


In [4]:
A = np.array([[0, 1.j],[-1.j, 0]])
print(A)
print(is_eq(dag(A), matInv(A)))

[[ 0.+0.j  0.+1.j]
 [-0.-1.j  0.+0.j]]
True


# Fusion Rules and F- and R- Symbol Calculation

## Fusion

Using the Verlinde formula to get S-matrix and then using that to extract the fusion rules $N_{\mathcal{A}\mathcal{B}}^{\mathcal{C}}$:

$\mathcal{S}_{\mathcal{A}\mathcal{B}} = \frac{1}{|G|}\sum_{h_i^A \in C_A, h_j^B \in C_B, h^A_i h^B_j = h^B_j h^A_i} \chi^A((x_i^A)^{-1}h_j^B x_i^A)\chi^B((x_j^B)^{-1}h_i^A x_j^B)$,

where $x_i^A h_1^A(x_i^A)^{-1} = h_1^A$ which is a $C_A$ representative [alt. $q_c r \bar{q}_c = c$].

$N_{\mathcal{A}\mathcal{B}}^{\mathcal{C}} = \sum_\mathcal{L} \frac{\mathcal{S}_{\mathcal{A}\mathcal{L}}\mathcal{S}_{\mathcal{B}\mathcal{L}}\mathcal{S}_{\mathcal{C}\mathcal{L}}^*}{\mathcal{S}_{\mathcal{0}\mathcal{L}}}$.

## F- and R-

Imagine we hace a space $\mathcal{H} = \mathcal{H}_b \otimes \mathcal{H}_a$ we can project down to the space of irreducible representations by $\mathcal{P}_{c}^{ba}(\ket{c, \mu}) \propto \sum_{h \in G}\sum_{z \in Z(r)}R^\dag_{\mu\mu}(z) B_h^b A_{q_c z \bar{q}_c}^b \otimes B_{h^{-1}c}^a A_{q_c z \bar{q}_c}^a$, where $\ket{c, \mu} \in \mathcal{H}_c$.

This allows us to define the splitting tree map: $\phi_c^{ba}: \mathcal{H}_c \rightarrow \mathcal{H}_b \otimes \mathcal{H}_a$ as $\phi_c^{ba}(\ket{c, \mu}) = \text{eig}(\mathcal{P}_{c}^{ba}(\ket{c, \mu}), \lambda = 1) \times e^{i \phi(c, \mu)}$. The phase being set by requiring correct group action of $A_g^{ba} \circ \phi_c^{ba} \circ A_{\bar{g}}^{c} = \phi_c^{ba} $.

The braiding action $\mathcal{B}_{ab}: \mathcal{H}_a\otimes\mathcal{H}_b\rightarrow\mathcal{H}_b\otimes\mathcal{H}_a$ is given by: $Flip \circ (\sum_g A_g^a \otimes B_g^b)$.

Combining the two facts we can evaluate the R-symbol as: $\mathcal{R}_c^{ab} \propto (\phi_c^{ba}(\ket{c, \mu}))^\dag\mathcal{B}_{ab}(\phi_c^{ab}(\ket{c, \mu}))$, with condition $|\mathcal{R}_c^{ab}| = 1$.

### F- Symbol

In a similar manner we can calculate the F-symbol as : $(\mathcal{F}_a^{bcd})_{xy} \propto (\phi_y^{cd} \circ \phi_a^{by}(\ket{c,\mu}))^\dag\phi_x^{bc} \circ \phi_a^{xd}(\ket{c,\mu})$

## $S_3$

### Fusion Rule Calculations and Group Def

In [5]:
G = np.zeros([3,3,6], complex)
G[:,:,0] = np.eye(3) # The S_3 group via perm rep
G[:,:,1] = np.array([[0, 1, 0],[1, 0, 0],[0, 0, 1]])
G[:,:,2] = np.array([[1, 0, 0],[0, 0, 1],[0, 1, 0]]) # sigma
G[:,:,3] = np.array([[0, 0, 1],[0, 1, 0],[1, 0, 0]])
G[:,:,4] = np.array([[0, 0, 1],[1, 0, 0],[0, 1, 0]]) # mu
G[:,:,5] = np.array([[0, 1, 0],[0, 0, 1],[1, 0, 0]])
g_lab = ['e','12','23','13','123','132']
g_lab_decomp = ['e','mm s m','s','m s mm','m','s m s']
order = len(g_lab)

C_e = np.array([0])  # Conj Classes
C_s = np.array([1, 2, 3]) # 2 is the rep
C_c = np.array([4, 5])
# i - 1 ; j - 2 ; k - 3 !!!

Z_e = np.array([0, 1, 2, 3, 4, 5])  # Centers Classes
Z_s = np.array([0, 2])
Z_c = np.array([0, 4, 5])

X_e = np.array([0])  # Centers Classes
X_s = np.array([3, 0, 1])
X_c = np.array([0, 2])

om = np.exp(1.j*2*np.pi/3) # chosen 3rd root of unity

def q(c):
    Q_C = [0, 3, 0, 1, 0, 2] # This for q_c instead
    return Q_C[c]

def grp_i(i):
    return matInv(G[:,:,i])

def index(h):
    for i in range(order): # to be generalised
        if np.array_equal(h, G[:,:,i]):
            return i

def inv(x):
    for i in range(8): # to be generalised
        if np.array_equal(matInv(G[:,:,x]), G[:,:,i]):
            return i

def grp(x):
    return G[:,:, x] # USE G AS A GLOBAL VAR!!! ---- CAREFUL !

In [6]:
print(is_eq(grp(2)@grp(4)@grp_i(2), grp(5)))
print(is_eq(grp(2)@grp(5)@grp_i(2), grp(4)))
print(is_eq(grp(2)@grp(1)@grp_i(2), grp(3)))
print(is_eq(grp(2)@grp(3)@grp_i(2), grp(1)))
print(is_eq(grp(4)@grp(2)@grp_i(4), grp(3)))
print(is_eq(grp_i(4)@grp(2)@grp(4), grp(1)))
print(is_eq(grp(4)@grp(5), grp(0)))
print(is_eq(grp(4)@grp(4), grp(5)))

True
True
True
True
True
True
True
True


In [7]:
# Def the Reps as Mats
def Triv(x):
    return np.array([[1]])
def Sgn(x):
    if (x in C_s):
        return np.array([[-1]])
    elif (x in C_e):
        return np.array([[1]])
    else:
        return np.array([[1]])
def E(x):
    sigma = np.array([[0, 1],[1, 0]])
    mu = np.array([[om, 0],[0, 1/om]])
    if x == 0:
        return np.eye(2)
    elif x == 1:
        return mu@mu@sigma@mu
    elif x == 2:
        return sigma
    elif x == 3:
        return mu@sigma@mu@mu
    elif x == 4:
        return mu
    else:
        return mu@mu
def Sgn_2(x):
    if (x in C_s):
        return np.array([[-1]])
    elif (x in C_e):
        return np.array([[1]])
    else:
        return np.array([[0]])
def Om_1(x):
    if x in [0]:
        return np.array([[1]])
    elif x in [4]:
        return np.array([[om]])
    elif x in [5]:
        return np.array([[1/om]])
    else:
        return np.array([[0]])
def Om_2(x):
    if x in [0]:
        return np.array([[1]])
    elif x in [4]:
        return np.array([[1/om]])
    elif x in [5]:
        return np.array([[om]])
    else:
        return np.array([[0]])

In [8]:
# Def chars of Reps
def triv(x):
    return 1
def sgn(x):
    if (x in C_s):
        return -1
    elif (x in C_e):
        return 1
    else:
        return 1
def e(x):
    if (x in C_s):
        return 0
    elif (x in C_e):
        return 2
    else:
        return -1
def sgn_2(x):
    if (x in C_s):
        return -1
    elif (x in C_e):
        return 1
    else:
        return 0
def om_1(x):
    if x in [0]:
        return 1
    elif x in [4]:
        return om
    elif x in [5]:
        return 1/om
    else:
        return 0
def om_2(x):
    if x in [0]:
        return 1
    elif x in [4]:
        return 1/om
    elif x in [5]:
        return om
    else:
        return 0


In [9]:
# Def some AUX Info about the reps of D(S3)
Z_s_Q = [Z_e, Z_e, Z_e, Z_s, Z_s, Z_c, Z_c, Z_c] # Don't mind the _Q for Q8
Dim_s_Q = [1,1,2,3,3,2,2,2]
C_mods_Q = [1,1,1,3,3,2,2,2]
chi_mods_Q = [1,1,2,1,1,1,1,1]
Z_reps_Q = [Triv, Sgn, E, Triv, Sgn_2, Triv, Om_1, Om_2]
C_s_Q = [C_e, C_e, C_e, C_s, C_s, C_c, C_c, C_c]
Z_charges = [triv, sgn, e, triv, sgn_2, triv, om_1, om_2]
Particles_lab_Q = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
p_num = 8
N_anyons = p_num
Particles_Q = [(i, Dim_s_Q[i], C_s_Q[i],  C_mods_Q[i], Z_s_Q[i], Z_charges[i], Z_reps_Q[i], chi_mods_Q[i], Particles_lab_Q[i]) for i in range(N_anyons)]

In [10]:
# Def Particles and some Fns

S = np.zeros([p_num,p_num], complex)
for i, _, C_i, _, _, char_i, _, _,_ in Particles_Q:
    for j, _, C_j, _, _, char_j, _, _,_ in Particles_Q:
        s = 0 
        for i_class, elem_i_class in enumerate(C_i):
            for j_class, elem_j_class in enumerate(C_j):
                if is_eq(grp(elem_i_class)@grp(elem_j_class), grp(elem_j_class)@grp(elem_i_class)):
                    s+=char_i(index(grp_i(q(elem_i_class))@grp(elem_j_class)@grp(q(elem_i_class))))*char_j(index(grp_i(q(elem_j_class))@grp(elem_i_class)@grp(q(elem_j_class))))
        S[i,j] = s # maybe conj

S_Q = S/order
# print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
#      for row in S]))
N_Q = np.zeros([p_num,p_num,p_num])
for I in range(p_num):
    for J in range(p_num):
        for K in range(p_num):
            for L in range(p_num):
                N_Q[I,J,K] += np.real(S_Q[I,L]*S_Q[J,L]*np.conj(S_Q[K,L])/S_Q[0,L])
N_Q = np.round(N_Q,1)

Query the S matrix:

In [11]:
an_1 = 4
an_2 = 5
print(Particles_lab_Q[an_1],',', Particles_lab_Q[an_2])
print('S_{aa}='+str(S_Q[an_1, an_1]))
print('S_{bb}='+str(S_Q[an_2, an_2]))
print('S_{ab}='+str(S_Q[an_1, an_2]))
print('S_{ba}='+str(S_Q[an_2, an_1]))
print('S_{0a}='+str(S_Q[0, an_2]))
print('S_{a0}='+str(S_Q[an_2, 0]))
print('S_{0a}='+str(S_Q[0, an_1]))
print('S_{a0}='+str(S_Q[an_1, 0]))

E , F
S_{aa}=(0.5+0j)
S_{bb}=(0.6666666666666666+0j)
S_{ab}=0j
S_{ba}=0j
S_{0a}=(0.3333333333333333+0j)
S_{a0}=(0.3333333333333333+0j)
S_{0a}=(0.5+0j)
S_{a0}=(0.5+0j)


Query the fusion rules:

In [12]:
i_1 = 6 # Prints fusion rules
i_2 = 6
print(Particles_lab_Q[i_1],'x', Particles_lab_Q[i_2], '=')
for i, indicator in enumerate(N_Q[i_1,i_2,:]):
    if np.round(indicator,1) != 0:
        print(np.round(indicator,1),'x', Particles_lab_Q[i])
S_r = 8*np.real(S)

G x G =
1.0 x A
1.0 x B
1.0 x G


### Defining $D(Q_8)$ representation matrices



#### $A_g:$

In [13]:
def a_mat(an_type, g):
    # Basis is |c \mu> 
    (id, d, C, C_mod, Z_set, char, Z_rep, chi_mod, lab) = Particles_Q[an_type]
    # use q(c) instead of X set
    A = np.zeros([d, d], dtype=complex)
    for i_c, c in enumerate(C):
        for mu in range(chi_mod):
            for i_cp, c_prim in enumerate(C):
                for nu in range(chi_mod):
                    if is_eq(grp(c_prim), grp(g)@grp(c)@grp_i(g)):
                        rep_map = Z_rep(index( grp_i(q(c_prim))@grp(g)@grp(q(c)) ))
                        mat_elem = rep_map[mu,nu]
                        A[i_cp*chi_mod+nu, i_c*chi_mod+mu]=mat_elem
    return A


#### $B_h:$

In [14]:
def b_mat(an_type, g):
    # Basis is |c \mu> 
    (id, d, C, C_mod, Z_set, char, Z_rep, chi_mod, lab) = Particles_Q[an_type]
    # use q(c) instead of X set
    B = np.zeros([d, d], dtype=complex)
    for i_c, c in enumerate(C):
        for mu in range(chi_mod):        
            if g == c:
                B[i_c*chi_mod+mu, i_c*chi_mod+mu]=1
    return B

#### Query the Matrices:

In [15]:
def mat_2d_lab(x):
    if np.size(x) == 4:
        if is_eq(x, np.array([[1+0j,0j],[0j,1+0j]])):
            return "E_2"
        if is_eq(x, np.array([[-1+0j,0j],[0j,-1+0j]])):
            return "-E_2"
        if is_eq(x, np.array([[1+0j,0j],[0j,-1+0j]])):
            return "Z"
        if is_eq(x, np.array([[-1+0j,0j],[0j,1+0j]])):
            return "-Z"
        if is_eq(x, np.array([[0j,1+0j],[1+0j,0j]])):
            return "X"
        if is_eq(x, np.array([[0j,-1+0j],[-1+0j,0j]])):
            return "-X"
        if is_eq(x, np.array([[0j,1+0j],[-1+0j,0j]])):
            return "jY"
        if is_eq(x, np.array([[0j,-1+0j],[1+0j,0j]])):
            return "-jY"
        if is_eq(x, np.array([[0j,0j],[0j,1+0j]])):
            return "Pd"
        if is_eq(x, np.array([[1+0j,0j],[0j,0j]])):
            return "Pu"
        if is_eq(x, np.array([[0j,0j],[0j,0j]])):
            return "0_2" 
        if is_eq(x, np.array([[-0.-1.j,  0.+0.j],[ 0.+0.j,  0.+1.j]])):
            return "-jZ"
        if is_eq(x, np.array([[-0.+1.j,  0.+0.j],[ 0.+0.j,  0.-1.j]])):
            return "jZ"
        if is_eq(x, np.array([[ 0.+0.j,  0.+1.j],[-0.-1.j,  0.+0.j]])):
            return "-Y"
        if is_eq(x, np.array([[ 0.+0.j,  0.-1.j],[-0.+1.j,  0.+0.j]])):
            return "Y"
        if is_eq(x, a_mat(2,1)):
            return "X e^(i 2\pi/3 Z)"
        if is_eq(x, a_mat(2,3)):
            return "X e^(-i 2\pi/3 Z)"
        if is_eq(x, a_mat(2,4)):
            return "e^(i 2\pi/3 Z)"
        if is_eq(x, a_mat(2,5)):
            return "e^(-i 2\pi/3 Z)"
    if np.size(x) == 1:
        if is_eq(x, np.array([[1]])):
            return "1"
        if is_eq(x, np.array([[0]])):
            return "0" # maybe dif 2d and 1d 0?
        if is_eq(x, np.array([[-1]])):
            return "-1"
    if np.size(x) == 9:
        return np.array2string(x)

    return "Error"

import pandas as pd

In [16]:
a_mat(4,4)

array([[ 0.+0.j,  0.+0.j,  1.+0.j],
       [-1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j]])

In [17]:
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [18]:
np.size(np.array([[2,2 ],[3,2]]))

4

In [19]:
data = {g_lab[m]:list(itertools.chain(*zip([mat_2d_lab(a_mat(i,m)) for i in range(N_anyons)],[mat_2d_lab(b_mat(i,m)) for i in range(N_anyons)]))) for m in range(6)}
data_a = {**{"A_g":[Particles_lab_Q[j] for j in range(N_anyons)]},**{g_lab[m]:[mat_2d_lab(a_mat(i,m)) for i in range(N_anyons)] for m in range(6)}}
data_b = {**{"B_h":[Particles_lab_Q[j] for j in range(N_anyons)]},**{g_lab[m]:[mat_2d_lab(b_mat(i,m)) for i in range(N_anyons)] for m in range(6)}}

mindex = pd.MultiIndex.from_product([[Particles_lab_Q[j] for j in range(N_anyons)],["A_g", "B_h"]], names=["first", "second"])
df = pd.DataFrame(data, index=mindex)
df_a = pd.DataFrame(data_a)
df_b = pd.DataFrame(data_b)

#print(df_a.to_latex(index=False))
display_side_by_side(df_a)

,A_g,e,12,23,13,123,132
0,A,1,1,1,1,1,1
1,B,1,-1,-1,-1,1,1
2,C,E_2,X e^(i 2\pi/3 Z),X,X e^(-i 2\pi/3 Z),e^(i 2\pi/3 Z),e^(-i 2\pi/3 Z)
3,D,[[1.+0.j 0.+0.j 0.+0.j]\n [0.+0.j 1.+0.j 0.+0.j]\n [0.+0.j 0.+0.j 1.+0.j]],[[1.+0.j 0.+0.j 0.+0.j]\n [0.+0.j 0.+0.j 1.+0.j]\n [0.+0.j 1.+0.j 0.+0.j]],[[0.+0.j 0.+0.j 1.+0.j]\n [0.+0.j 1.+0.j 0.+0.j]\n [1.+0.j 0.+0.j 0.+0.j]],[[0.+0.j 1.+0.j 0.+0.j]\n [1.+0.j 0.+0.j 0.+0.j]\n [0.+0.j 0.+0.j 1.+0.j]],[[0.+0.j 0.+0.j 1.+0.j]\n [1.+0.j 0.+0.j 0.+0.j]\n [0.+0.j 1.+0.j 0.+0.j]],[[0.+0.j 1.+0.j 0.+0.j]\n [0.+0.j 0.+0.j 1.+0.j]\n [1.+0.j 0.+0.j 0.+0.j]]
4,E,[[1.+0.j 0.+0.j 0.+0.j]\n [0.+0.j 1.+0.j 0.+0.j]\n [0.+0.j 0.+0.j 1.+0.j]],[[-1.+0.j 0.+0.j 0.+0.j]\n [ 0.+0.j 0.+0.j 1.+0.j]\n [ 0.+0.j 1.+0.j 0.+0.j]],[[ 0.+0.j 0.+0.j -1.+0.j]\n [ 0.+0.j -1.+0.j 0.+0.j]\n [-1.+0.j 0.+0.j 0.+0.j]],[[ 0.+0.j 1.+0.j 0.+0.j]\n [ 1.+0.j 0.+0.j 0.+0.j]\n [ 0.+0.j 0.+0.j -1.+0.j]],[[ 0.+0.j 0.+0.j 1.+0.j]\n [-1.+0.j 0.+0.j 0.+0.j]\n [ 0.+0.j -1.+0.j 0.+0.j]],[[ 0.+0.j -1.+0.j 0.+0.j]\n [ 0.+0.j 0.+0.j -1.+0.j]\n [ 1.+0.j 0.+0.j 0.+0.j]]
5,F,E_2,X,X,X,E_2,E_2
6,G,E_2,X e^(-i 2\pi/3 Z),X,X e^(i 2\pi/3 Z),e^(i 2\pi/3 Z),e^(-i 2\pi/3 Z)
7,H,E_2,X e^(i 2\pi/3 Z),X,X e^(-i 2\pi/3 Z),e^(-i 2\pi/3 Z),e^(i 2\pi/3 Z)


In [20]:
a_mat(3, 1)

array([[1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j]])

### The fusion projectors and plitting tree maps

In [21]:
def fusion_products(an_1, an_2):
    res = []
    for i, indicator in enumerate(N_Q[an_1,an_2,:]):
        if indicator != 0:
            res += [i]
    return res

def do_they_fuse(an_1, an_2, an_3):
    return an_3 in fusion_products(an_1, an_2)

def proj_norm(P):
    a = np.trace(P)
    b = np.trace(P@P)
    return P*a/b

def splitting_tree(an_1, an_2, an_3, id): #Only good if id = 0
    Pro = rep_proj_spec(an_1, an_2, an_3, id)
    w, v = np.linalg.eig(Pro)
    for i_lam, lam in enumerate(w):
        if np.round(lam, 2) == 1:
            Vec = v[:,i_lam]
    return Vec

def splitting_tree_proper(an_1, an_2, an_3): #Returns a properlt gauge fixed basis
    (_, d_3, _, _, _, _, _, _, _) = Particles_Q[an_3]
    (_, d_2, _, _, _, _, _, _, _) = Particles_Q[an_2]
    (_, d_1, _, _, _, _, _, _, _) = Particles_Q[an_1]
    Basis_unfixed = []
    order = np.size(G[0,0,:])
    for i_3 in range(d_3):
        Basis_unfixed += [splitting_tree(an_1, an_2, an_3, i_3)]
    Basis_fixed = Basis_unfixed
    if d_3 == 1:
        return Basis_fixed
    else:
        for i_3 in range(1,d_3):
            not_done = 1
            for g in range(order):
                if not_done:
                    Vec_3 = np.zeros([d_1*d_2], dtype=complex)
                    for mat_mul in range(d_3):
                        Vec_3 += a_mat(an_3, g)[0,mat_mul]*Basis_unfixed[mat_mul]
                    Vec_12 = np.kron(a_mat(an_1, g), a_mat(an_2, g))@Basis_unfixed[0]
                    overlap_12 = np.vdot(Basis_unfixed[i_3], Vec_3)
                    overlap_3 = np.vdot(Basis_unfixed[i_3], Vec_12)
                    if overlap_12 != 0 and overlap_3 != 0:
                        Basis_fixed[i_3] = Basis_unfixed[i_3]*overlap_12/overlap_3
                        not_done = 0
    return Basis_fixed




def rep_proj(an_1, an_2, an_3):
    (id_3, _, C_3, _, Z_3, char_3, Z_rep_3, chi_mod_3, _) = Particles_Q[an_3]
    (_, d_2, _, _, _, _, _, _, _) = Particles_Q[an_2]
    (_, d_1, _, _, _, _, _, _, _) = Particles_Q[an_1]
    Res = np.zeros([d_1*d_2,d_1*d_2], dtype=complex)
    for c in C_3:
        for z in Z_3:
            for h in range(order):
                Res += np.conjugate(char_3(z))*np.kron( b_mat(an_1, index( grp_i(h)@grp(c) ) ) @ a_mat(an_1,index( grp(q(c))@grp(z)@grp_i(q(c)) )),
                b_mat(an_2,h) @ a_mat(an_2,index( grp(q(c))@grp(z)@grp_i(q(c)) )) )
    return proj_norm(Res) # Normalisation issues :(

def rep_proj_spec(an_1, an_2, an_3, id):# Basis is |c \mu> # NOT SURE if the FORM is correct 
    (id_3, _, C_3, _, Z_3, char_3, Z_rep_3, chi_mod_3, _) = Particles_Q[an_3]
    (_, d_2, _, _, _, _, _, _, _) = Particles_Q[an_2]
    (_, d_1, _, _, _, _, _, _, _) = Particles_Q[an_1]
    Res = np.zeros([d_1*d_2,d_1*d_2], dtype=complex)
    i_c = int(np.floor(id/chi_mod_3))
    c = C_3[i_c]
    mu = np.mod(id,chi_mod_3)
    for z in Z_3:
        for h in range(order):
            Res += Z_rep_3(z)[mu,mu].conj()*np.kron( b_mat(an_1, # Investigate does this one work???
            index( grp_i(h)@grp(c) ) ) @ a_mat(an_1,
            index( grp(q(c))@grp(z)@grp_i(q(c)) )),
            b_mat(an_2,h) @ a_mat(an_2,index( grp(q(c))@grp(z)@grp_i(q(c)) )) )
    return proj_norm(Res) # Normalisation issues :(



In [22]:
np.size(G[0,0,:])

6

### The R-sym Calculation:

In [23]:
def r_symbol(an_1, an_2, an_3):
    (_, d_2, _, _, _, _, _, _, _) = Particles_Q[an_2]
    (_, d_1, _, _, _, _, _, _, _) = Particles_Q[an_1]
    T = np.zeros([d_1*d_2, d_1*d_2], dtype=complex)
    # T_k = np.zeros([d_1*d_2, d_1*d_2], dtype=complex)
    for g in range(order):
        T += np.kron(a_mat(an_1,g), b_mat(an_2,g))
        # T_k += np.kron(b_mat(an_2,g), a_mat(an_1,g))
    # print('apply', T)
    F = np.zeros([d_1*d_2, d_1*d_2])
    for i in range(d_1):
        for j in range(d_2):
            for m in range(d_2):
                for n in range(d_1):
                    if i == n and j == m:
                        F[m*d_1+n, i*d_2 + j] = 1

    # Contrcut the Splitting Tree Basis!
    Vec_1 = splitting_tree(an_2, an_1, an_3,0)
    Vec_2 = splitting_tree(an_1, an_2, an_3,0) 
    Vec_2 = F@T@Vec_2 # The other way gives 0
    # print(Vec_1)
    # print(Vec_2)
    overlap = np.round(np.vdot(Vec_1, Vec_2),2) # issue !!!!
    # print(Vec_1, Vec_2)
    return overlap
    
def fusion_and_braid(an_1, an_2): # Depends on a lot of outside variables!!!
    # print(Particles_lab_Q[an_1],'x', Particles_lab_Q[an_2], '=')
    # Reduce text
    pr = []
    for i, indicator in enumerate(N_Q[an_1,an_2,:]):
        if np.round(indicator,1) != 0:
            # Calculate the R-Symbol
            #print(int(indicator),'x', Particles_lab_Q[i], 'with R_symb', str(r_symbol(an_1,an_2, i)))
            pr += [(i, r_symbol(an_1,an_2, i))]
    print(an_1, 'x', an_2, '=', pr)
    return 0

The Results, quesries of fusion and R-symbols:

In [24]:
fusion_and_braid(6,6)

6 x 6 = [(0, (-0.5-0.87j)), (1, (0.5+0.87j)), (6, (-0.5+0.87j))]


0

In [25]:
for an_1 in range(p_num):
    for an_2 in range(p_num):
        fusion_and_braid(an_1, an_2)

0 x 0 = [(0, (1+0j))]
0 x 1 = [(1, (1+0j))]
0 x 2 = [(2, (1+0j))]
0 x 3 = [(3, (1+0j))]
0 x 4 = [(4, (1+0j))]
0 x 5 = [(5, (1+0j))]
0 x 6 = [(6, (1+0j))]
0 x 7 = [(7, (1+0j))]
1 x 0 = [(1, (1+0j))]
1 x 1 = [(0, (1+0j))]
1 x 2 = [(2, (1+0j))]
1 x 3 = [(4, (-1+0j))]
1 x 4 = [(3, (-1+0j))]
1 x 5 = [(5, (1+0j))]
1 x 6 = [(6, (1+0j))]
1 x 7 = [(7, (1+0j))]
2 x 0 = [(2, (1+0j))]
2 x 1 = [(2, (1+0j))]
2 x 2 = [(0, (1+0j)), (1, (-1+0j)), (2, (1+0j))]
2 x 3 = [(3, (1-0j)), (4, (-1+0j))]
2 x 4 = [(3, (-1+0j)), (4, (1-0j))]
2 x 5 = [(6, (-0.5+0.87j)), (7, (-0.5-0.87j))]
2 x 6 = [(5, (-0.5-0.87j)), (7, (-0.5+0.87j))]
2 x 7 = [(5, (-0.5+0.87j)), (6, (-0.5-0.87j))]
3 x 0 = [(3, (1+0j))]
3 x 1 = [(4, (1+0j))]
3 x 2 = [(3, (1-0j)), (4, (1-0j))]
3 x 3 = [(0, (1+0j)), (2, (1+0j)), (5, (1+0j)), (6, (-0.5-0.87j)), (7, (-0.5+0.87j))]
3 x 4 = [(1, (1+0j)), (2, (1+0j)), (5, (-1+0j)), (6, (0.5+0.87j)), (7, (0.5-0.87j))]
3 x 5 = [(3, (1+0j)), (4, (1+0j))]
3 x 6 = [(3, (1-0j)), (4, (1-0j))]
3 x 7 = [(3, (1+0j))

### F- Symbols 


In [26]:
print(fusion_products(5,5))
print(do_they_fuse(5,5,5))
np.zeros([2])

[0, 1, 5]
True


array([0., 0.])

In [27]:
def f_mat(an_a, an_b, an_c, an_d):
    (_, d_a, _, _, _, _, _, _, _) = Particles_Q[an_a]
    (_, d_b, _, _, _, _, _, _, _) = Particles_Q[an_b]
    (_, d_c, _, _, _, _, _, _, _) = Particles_Q[an_c]
    (_, d_d, _, _, _, _, _, _, _) = Particles_Q[an_d]
    x_ids = []
    y_ids = []
    # print('x=')
    for x in fusion_products(an_b, an_c):
        if do_they_fuse(x,an_d,an_a):
            x_ids+=[x]
            # print(Particles_lab_Q[x])
    # print('y=')
    for y in fusion_products(an_c, an_d):
        if do_they_fuse(an_b,y,an_a):
            y_ids+=[y]
            # print(Particles_lab_Q[y])
    F = np.zeros([len(x_ids), len(y_ids)], dtype=complex)

    for i_x, x in enumerate(x_ids):
        for i_y, y in enumerate(y_ids):
            (_, d_x, _, _, _, _, _, _, _) = Particles_Q[x]
            (_, d_y, _, _, _, _, _, _, _) = Particles_Q[y]
            i_vec_a = 0
            vec_xd = splitting_tree(x, an_d, an_a, i_vec_a)
            vec_by = splitting_tree(an_b, y, an_a, i_vec_a)
            vec_bcd_via_x = np.zeros([d_d*d_b*d_c], dtype=complex)
            vec_bcd_via_y = np.zeros([d_d*d_b*d_c], dtype=complex)
            x_basis = splitting_tree_proper(an_b, an_c, x)
            y_basis = splitting_tree_proper(an_c, an_d, y)
            for id_x in range(d_x):
                for id_d in range(d_d):
                    e_d = np.zeros([d_d])
                    e_d[id_d] = 1
                    vec_bcd_via_x += vec_xd[id_x*d_d + id_d]*np.kron(x_basis[id_x] ,e_d)
            for id_y in range(d_y):
                for id_b in range(d_b):
                    e_b = np.zeros([d_b])
                    e_b[id_b] = 1
                    vec_bcd_via_y += vec_by[id_b*d_y + id_y]*np.kron(e_b,y_basis[id_y])
            F[i_x, i_y] += np.vdot(vec_bcd_via_y, vec_bcd_via_x)
    F = np.round(F, 2)
    # print('F=',F)
    return F

def f_mat_debug(an_a, an_b, an_c, an_d):
    (_, d_a, _, _, _, _, _, _, _) = Particles_Q[an_a]
    (_, d_b, _, _, _, _, _, _, _) = Particles_Q[an_b]
    (_, d_c, _, _, _, _, _, _, _) = Particles_Q[an_c]
    (_, d_d, _, _, _, _, _, _, _) = Particles_Q[an_d]
    x_ids = []
    y_ids = []
    print('x=')
    for x in fusion_products(an_b, an_c):
        if do_they_fuse(x,an_d,an_a):
            x_ids+=[x]
            print(Particles_lab_Q[x])
    print('y=')
    for y in fusion_products(an_c, an_d):
        if do_they_fuse(an_b,y,an_a):
            y_ids+=[y]
            print(Particles_lab_Q[y])
    F = np.zeros([len(x_ids), len(y_ids)], dtype=complex)

    for i_x, x in enumerate(x_ids):
        for i_y, y in enumerate(y_ids):
            (_, d_x, _, _, _, _, _, _, _) = Particles_Q[x]
            (_, d_y, _, _, _, _, _, _, _) = Particles_Q[y]
            i_vec_a = 0
            vec_xd = splitting_tree(x, an_d, an_a, i_vec_a)
            vec_by = splitting_tree(an_b, y, an_a, i_vec_a)
            vec_bcd_via_x = np.zeros([d_d*d_b*d_c], dtype=complex)
            vec_bcd_via_y = np.zeros([d_d*d_b*d_c], dtype=complex)
            x_basis = splitting_tree_proper(an_b, an_c, x)
            y_basis = splitting_tree_proper(an_c, an_d, y)
            for id_x in range(d_x):
                for id_d in range(d_d):
                    e_d = np.zeros([d_d])
                    e_d[id_d] = 1
                    vec_bcd_via_x += vec_xd[id_x*d_d + id_d]*np.kron(x_basis[id_x] ,e_d)
            for id_y in range(d_y):
                for id_b in range(d_b):
                    e_b = np.zeros([d_b])
                    e_b[id_b] = 1
                    vec_bcd_via_y += vec_by[id_b*d_y + id_y]*np.kron(e_b,y_basis[id_y])
            F[i_x, i_y] += np.vdot(vec_bcd_via_y, vec_bcd_via_x)
    F = np.round(F, 2)
    # print('F=',F)
    return F
    

In [28]:
def f_mat_elem(an_a, an_b, an_c, an_d, x, y, i_vec_a): # use to debug
    (_, d_a, _, _, _, _, _, _, _) = Particles_Q[an_a]
    (_, d_b, _, _, _, _, _, _, _) = Particles_Q[an_b]
    (_, d_c, _, _, _, _, _, _, _) = Particles_Q[an_c]
    (_, d_d, _, _, _, _, _, _, _) = Particles_Q[an_d]
    F = 0.j
    (_, d_x, _, _, _, _, _, _, _) = Particles_Q[x]
    (_, d_y, _, _, _, _, _, _, _) = Particles_Q[y]
    vec_xd = splitting_tree(x, an_d, an_a, i_vec_a)
    vec_by = splitting_tree(an_b, y, an_a, i_vec_a)
    vec_bcd_via_x = np.zeros([d_b*d_c*d_d], dtype=complex)
    vec_bcd_via_y = np.zeros([d_b*d_c*d_d], dtype=complex)
    x_basis = splitting_tree_proper(an_b, an_c, x)
    y_basis = splitting_tree_proper(an_c, an_d, y)
    for id_x in range(d_x):
        for id_d in range(d_d):
            e_d = np.zeros([d_d])
            e_d[id_d] = 1
            print(len(vec_bcd_via_x), len(vec_xd), len(x_basis[id_x]), len(e_d), d_a, d_b, d_c, d_d)
            vec_bcd_via_x += vec_xd[id_x*d_d + id_d]*np.kron(x_basis[id_x] ,e_d)
    for id_y in range(d_y):
        for id_b in range(d_b):
            e_b = np.zeros([d_b])
            e_b[id_b] = 1
            print(len(vec_bcd_via_y), len(vec_by), len(y_basis[id_y]), len(e_b), d_a, d_b, d_c, d_d)
            vec_bcd_via_y += vec_by[id_b*d_y + id_y]*np.kron(e_b, y_basis[id_y])
    print('x up', np.round(np.real(vec_bcd_via_x),2))
    print('y down',np.round(np.real(vec_bcd_via_y), 2))
    F += np.vdot(vec_bcd_via_x, vec_bcd_via_y)
    F = np.round(F, 2)
    return F

    

Query the F-matrices

In [29]:
F = f_mat(6, 6, 6, 6)

In [30]:
np.round(F@dag(F), 2)

array([[ 0.99-0.j, -0.01-0.j,  0.  +0.j],
       [-0.01-0.j,  0.99-0.j,  0.  +0.j],
       [ 0.  +0.j,  0.  +0.j,  0.99+0.j]])

### F- and R- Symbol Consistency check?

In [31]:
def get_all_F_symbols():
    Res = np.zeros([p_num, p_num, p_num, p_num, p_num, p_num], dtype=complex) # F^{bcd}_{axy}
    for an_a in range(p_num):
        for an_b in range(p_num):
            for an_c in range(p_num):
                for an_d in range(p_num):
                    x_ids = []
                    y_ids = []
                    for x in fusion_products(an_b, an_c):
                        if do_they_fuse(x,an_d,an_a):
                            x_ids+=[x]
                    for y in fusion_products(an_c, an_d):
                        if do_they_fuse(an_b,y,an_a):
                            y_ids+=[y]
                    if len(x_ids) != 0 and len(y_ids) != 0:
                        # print(an_a,an_b,an_c,an_d,x,y)
                        to_fill = f_mat(an_a, an_b, an_c, an_d)
                        for i_y, y in enumerate(y_ids):
                            for i_x, x in enumerate(x_ids):
                                Res[an_b, an_c, an_d, an_a, x, y] = to_fill[i_x, i_y]
    return Res

def get_all_R_symbols():
    Res = np.zeros([p_num, p_num, p_num], dtype=complex) # R^{bc}_{a}
    for an_a in range(p_num):
        for an_b in range(p_num):
            for an_c in range(p_num):
                if do_they_fuse(an_b, an_c, an_a):
                    Res[an_b, an_c, an_a] = r_symbol(an_b,an_c, an_a)
    return Res

In [32]:
F_s_Q = get_all_F_symbols()
R_s_Q = get_all_R_symbols()

KeyboardInterrupt: 

In [ ]:
def pentagon_check():
    # abcdefgkl
    LHS = np.zeros([p_num,p_num,p_num,p_num,p_num,p_num,p_num,p_num,p_num], dtype= complex)
    RHS = np.zeros([p_num,p_num,p_num,p_num,p_num,p_num,p_num,p_num,p_num], dtype= complex)
    for a in range(p_num):
        for b in range(p_num):
            for c in range(p_num):
                for d in range(p_num):
                    for e in range(p_num):
                        for f in range(p_num):
                            for g in range(p_num):
                                for k in range(p_num):
                                    for l in range(p_num):
                                        LHS[a,b,c,d,e,f,g,k,l] = F_s_Q[f,c,d,e,g,l]*F_s_Q[a,b,l,e,f,k]
    for a in range(p_num):
        for b in range(p_num):
            for c in range(p_num):
                for d in range(p_num):
                    for e in range(p_num):
                        for f in range(p_num):
                            for g in range(p_num):
                                for k in range(p_num):
                                    for l in range(p_num):
                                        for h in range(p_num):
                                            RHS[a,b,c,d,e,f,g,k,l] += F_s_Q[a,b,c,g,f,h]*F_s_Q[a, h, d, e, g, k]*F_s_Q[b,c,d,k,h,l]
    return np.concatenate([np.abs(np.reshape(LHS-RHS, [p_num**9, 1])),np.abs(np.reshape((LHS+RHS)/2, [p_num**9, 1]))], axis=1)

In [ ]:
result = pentagon_check()

In [ ]:
print(result[:,1])

[1. 0. 0. ... 0. 0. 0.]


In [ ]:
fails = result[result[:,0]!=0, :]

In [ ]:
result[47967933,1]

0.5020626713757954